In [ ]:
import numpy as np ## use import cupy as np for GPU, should be compatible but not tested
import pandas as pd
from scipy import stats

def vectorised_lstsq(A, b):
    """
    Vectorised least squares solution for multiple linear regression."

    Parameters
    ----------
    A : array_like
        The design matrix of shape (..., m, n), where m is the number of observations and n is the number of predictors.
    b : array_like
        The response vector of shape (..., m).

    Returns
    -------
    x : array_like
        The estimated coefficients of shape (..., n).
    rss : array_like
        The residual sum of squares of shape (...).
    dof : array_like
        The degrees of freedom of shape (...).
    se : array_like
        The standard errors of the estimated coefficients of shape (..., n).
    """
    m = A.shape[-2]  # number of observations
    n = A.shape[-1]  # number of predictors
    dof = m - n

    ATA = np.einsum('...mi,...mj->...ij', A, A)
    ATb = np.einsum('...mi,...m->...i', A, b)
    x = np.linalg.solve(ATA, ATb)
    residuals = b - np.einsum('...ij,...j->...i', A, x)
    rss = np.sum(np.square(residuals), axis = -1)
    sigma2 = rss / dof
    ATA_inv = np.linalg.inv(ATA)
    diagATAinv = np.diagonal(ATA_inv, axis1 = -2, axis2 = -1)
    se = np.sqrt(sigma2[..., np.newaxis] * diagATAinv)

    return x, rss, dof, se

def ht(thetas, ts):
    """
    Calculate the h_t function in the lp-ntPET model.

    Parameters
    ----------
    thetas : array_like
        The parameters of shape (3, n), where n is the number of basis functions.
        The parameters are [alpha, tD, tP].
    ts : array_like
        The time points of shape (m,), where m is the number of time points.
        The time points are in seconds.

    Returns
    -------
    array_like
        The h_t function values of shape (n, m).
        The h_t function is defined as:
        h_t = max(0, (t - tD) / (tP - tD)) ^ alpha * exp(alpha * (1 - (t - tD) / (tP - tD))) * (t > tD)
    """

    alphas, tDs, tPs = thetas
    ts = ts[np.newaxis, ...]
    alphas = alphas[..., np.newaxis]
    tDs = tDs[..., np.newaxis]
    tPs = tPs[..., np.newaxis]
    return np.maximum(0, (ts - tDs) / (tPs - tDs)) ** alphas * np.exp(alphas * (1 - (ts - tDs) / (tPs - tDs))) * (ts > tDs)

def get_optimal_basis(rss, thetas, params, dof, se):
    """
    Get the optimal basis for the lp-ntPET model.
    The optimal basis is the one that minimizes the residual sum of squares (rss).
    The optimal parameters are the ones that correspond to the minimum rss.
    The optimal parameters are returned as a concatenation of the optimal parameters and thetas.
    The optimal standard errors are also returned.
    The optimal standard errors are the ones that correspond to the minimum rss.

    Parameters
    ----------
    rss : array_like
        The residual sum of squares of shape (n, m), where n is the number of basis functions and m is the number of voxels.
    thetas : array_like
        The parameters of shape (q, n) from the basis function, where n is the number of basis function and q is the number of parameters.
        The parameters are [alpha, tD, tP].
    params : array_like
        The parameters of shape (n, m, p), where n is the number of parameters, m is the number of voxels and p is the number of parameters.
        The parameters are [R1, k2, k2a, gamma].
    dof : array_like
        The degrees of freedom, scalar.
    se : array_like
        The standard errors of the estimated coefficients of shape (n, m, p), where n is the number of parameters, m is the number of voxels and p is the number of parameters.
        The standard errors are [R1, k2, k2a, gamma].

    Returns
    -------
    return_params : array_like
        The optimal parameters of shape (m, p + 3), where m is the number of voxels and p is the number of parameters.
        The optimal parameters are [R1, k2, k2a, gamma, alpha, tD, tP].
    min_rss : array_like
        The minimum residual sum of squares of shape (m,), where m is the number of voxels.
    min_idx : array_like
        The index of the minimum residual sum of squares, scalar.
    optimal_se : array_like
        The optimal standard errors of the estimated coefficients of shape (m, p), where m is the number of voxels and p is the number of parameters.
        The optimal standard errors are [R1, k2, k2a, gamma].
    """
    min_rss = np.min(rss, axis = 0)
    min_idx = np.argmin(rss, axis = 0)
    optimal_thetas = thetas.T[min_idx]
    optimal_params = params[min_idx, np.arange(params.shape[1]), :]
    optimal_se = se[min_idx, np.arange(se.shape[1]), :]
    return_params = np.concatenate((optimal_params, optimal_thetas), axis = -1)
    return return_params, min_rss, dof, optimal_se

def get_BIC(rss, n_params, n_samples):
    """"
    Calculate the Bayesian Information Criterion (BIC) for the lp-ntPET model."

    Parameters
    ----------
    rss : array_like
        The residual sum of squares of shape (...).
    n_params : int
        The number of parameters in the model.
    n_samples : int
        The number of samples in the model.
        
    Returns
    -------
    float
        The BIC value.
    """
    p = n_params
    n = n_samples
    return n * np.log(rss / n) + p * np.log(n)

In [ ]:
data_path = "Voxel-wise-TACs-scale1-wholeBrain_reformatted.csv"

data = pd.read_csv(data_path)
dts = data.iloc[:, 0].to_numpy() ## time intervals
ts = data.iloc[:, 1].to_numpy() ## mid-time points
Cr = data.iloc[:, 2].to_numpy() ## reference TAC
Cts = data.iloc[:, 3:].to_numpy().T ## tissue TACs

w = None ## weight, not used in this example

## Basis functions
## Thetas are (alpha, tD, tP)
tDs = np.arange(-5, 50, 2.5)
alphas = np.array([0.25, 1, 4])
all_combinations = np.array([(alpha, tD, tP) 
                             for alpha in alphas
                             for tD in tDs 
                             for tP in np.arange(tD + 1.25, 55, 2.5)])
thetas = all_combinations.reshape(-1, 3).T
hts = ht(thetas, ts)

Cr_cumsum = np.cumsum(Cr) * dts
Cts_cumsum = np.cumsum(Cts, axis = -1) * dts

In [ ]:
## MRTM model

# Set up design matrix A where A*[R1, k2, k2a] = Ct
A = np.stack([np.broadcast_to(Cr, Cts_cumsum.shape), 
              np.broadcast_to(Cr_cumsum, Cts_cumsum.shape), 
              -Cts_cumsum
              ], axis = -1)

# Solve using least squares
if w is None:
    params_MRTM, rss_MRTM, dof_MRTM, se_MRTM = vectorised_lstsq(A, Cts)
else:
    W = np.diag(w)
    A_w = W @ A
    Cts_w = W @ Cts
    params_MRTM, rss_MRTM, dof_MRTM, se_MRTM = vectorised_lstsq(A_w, Cts_w)

In [ ]:
## lp-ntPET model

Bts = np.cumsum(Cts[None, ...] * hts[:, None, ...], axis = -1) * dts[None, None, ...]

# Set up design matrix A where A*[R1, k2, k2a, gamma] = Ct
A = np.stack([np.broadcast_to(Cr, Bts.shape), 
              np.broadcast_to(Cr_cumsum, Bts.shape), 
              np.broadcast_to(-Cts_cumsum, Bts.shape), 
              -Bts
              ], axis = -1)

# Solve using least squares
lp_nt = True
if w is None:
    params, rss, dof, se = vectorised_lstsq(A, Cts)
else:
    W = np.diag(w)
    A_w = W @ A
    Cts_w = W @ Cts
    params, rss, dof, se = vectorised_lstsq(A_w, Cts_w)

params_lp_nt, rss_lp_nt, dof_lp_nt, se_lp_nt = get_optimal_basis(rss, thetas, params, dof, se)

In [ ]:
# t-statistic for gamma
t_stat = params_lp_nt[..., 3] / se_lp_nt[..., 3]

# p-value (two-tailed) for gamma = 0
p_values = 2 * stats.t.sf(np.abs(t_stat), dof_lp_nt)

In [ ]:
# BIC for model comparison
num_of_data = Cts.shape[-1]
num_of_coef_MRTM = 3
num_of_coef_lp_nt = 7
BIC_MRTM = get_BIC(rss_MRTM, num_of_coef_MRTM, num_of_data)
BIC_lp_nt = get_BIC(rss_lp_nt, num_of_coef_lp_nt, num_of_data)
model = np.where(BIC_MRTM < BIC_lp_nt, "MRTM", "lp-ntPET")